In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
from gspx.datasets import uk_wind

df = uk_wind()

In [ ]:
import requests
import time
from tqdm import tqdm
key = "<write here your api key>"

data = dict()

for idx, row in tqdm(df.iterrows(), total=len(df)):
    def collect(d, field, new_value):
        if field not in d:
            d[field] = list()
        d[field].append(new_value)

    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    response = requests.get(
        "https://api.openweathermap.org/data/2.5/weather?"
        f"lat={lat}&lon={lon}&appid={key}")
    out = eval(response.text)
    fields = ['humidity', 'pressure', 'temp']
    for field in fields:
        collect(data, field, out['main'][field])
    collect(data, 'wind_speed', out['wind']['speed'])
    collect(data, 'wind_degrees', out['wind']['deg'])
    if idx % 10 == 0:
        time.sleep(10)

In [ ]:
import pandas as pd

# Final request at: Wed, 20 Apr 2022 13:06:30 GMT
df_weather = pd.concat((
    df.drop(columns=['WIND_SPEED', 'WIND_DEGREES']),
    pd.DataFrame(data)), axis=1)

df_weather = df_weather.rename(columns={c: c.lower() for c in df_weather.columns})
df_weather

In [ ]:
df_weather.to_csv(
    "/home/gbr/gspx/resources/uk_weather_at_20Apr202213pm.gz",
    sep="\t", compression='gzip', index=False)